In [1]:
import pandas as pd
import numpy as np
import folium

Download data from:  
<https://ffiec.cfpb.gov/data-publication/modified-lar/2021>

Make sure 2021 is selected and enter `B4TYDEB6GKMZO031MB27` as the LEI for Bank of America. Opt to "Include File Header" and then click on "Download Modified LAR with Header".

In [2]:
df = pd.read_csv('B4TYDEB6GKMZO031MB27_header.csv', delimiter='|')
print(len(df))
df.head()

368728


/var/folders/54/_v7k89551fvc9hxwrrdbw_lr0000gn/T/ipykernel_78154/2495106069.py:1: DtypeWarning: Columns (73) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('B4TYDEB6GKMZO031MB27_header.csv', delimiter='|')


,activity_year,lei,loan_type,loan_purpose,preapproval,construction_method,occupancy_type,loan_amount,action_taken,state_code,...,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose
0,2021,B4TYDEB6GKMZO031MB27,1,1,2,1,1,255000,1,DE,...,1,1,2,NaN,NaN,NaN,NaN,2,2,2
1,2021,B4TYDEB6GKMZO031MB27,1,1,2,1,1,655000,2,NV,...,1,1,7,NaN,NaN,NaN,NaN,2,2,2
2,2021,B4TYDEB6GKMZO031MB27,1,1,1,1,1,875000,8,NaN,...,1,1,7,NaN,NaN,NaN,NaN,2,2,2
3,2021,B4TYDEB6GKMZO031MB27,1,31,2,1,1,515000,1,CA,...,1,1,2,7.0,NaN,NaN,NaN,2,2,2
4,2021,B4TYDEB6GKMZO031MB27,1,31,2,1,1,665000,4,CA,...,1,1,6,NaN,NaN,NaN,NaN,2,2,2


In [3]:
df = df[['action_taken', 'census_tract']]
df = df.dropna()
df['census_tract'] = df['census_tract'].astype(int)
print(df.dtypes)
df['census_tract'] = df['census_tract'].astype(str)


df = df[(df['action_taken'] == 1) | (df['action_taken'] == 3)]
print(df['action_taken'].value_counts())
print(df['census_tract'].nunique())

print(len(df))

df.head()


action_taken    int64
census_tract    int64
dtype: object
1    172270
3    110301
Name: action_taken, dtype: int64
50647
282571


,action_taken,census_tract
0,1,10003015200
3,1,6059086502
5,1,26163596100
6,1,45051051604
9,1,9009165600


In [4]:
## subsample
# df = df.sample(n=120000)

## Retrieve Locations of Census Tracts from Census Gazetteer

Downloaded data from:  
<https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.2021.html>

Find `Census Tracts` and click on "Download the National Census Tracts Gazetteer Files".

In [5]:
loc_df = pd.read_csv('2021_Gaz_tracts_national.txt', delimiter='\t')
print(loc_df.dtypes)
# loc_df.head()
loc_df.rename(columns={ loc_df.columns[-1]: "INTPTLONG" }, inplace = True)
# loc_df.columns


USPS                                                                                                                                       object
GEOID                                                                                                                                       int64
ALAND                                                                                                                                       int64
AWATER                                                                                                                                      int64
ALAND_SQMI                                                                                                                                float64
AWATER_SQMI                                                                                                                               float64
INTPTLAT                                                                                                                    

In [6]:
tract2loc = dict(zip(loc_df['GEOID'], zip(loc_df['INTPTLAT'], loc_df['INTPTLONG'])))
# print(tract2loc[list(tract2loc.keys())[0]])



In [7]:
# print(df.head(1))
# print(df.dtypes)

df['location'] = df['census_tract'].astype(int).map(tract2loc)

df[['lat', 'lon']] = pd.DataFrame(df['location'].tolist(), index=df.index)

# print(df.iloc[0])

df = df.dropna()
print(len(df))
print(df['action_taken'].value_counts())
df.head()


206418
1    127286
3     79132
Name: action_taken, dtype: int64


,action_taken,census_tract,location,lat,lon
0,1,10003015200,"(39.7070504, -75.5832416)",39.707050,-75.583242
3,1,6059086502,"(33.8503559, -117.9121351)",33.850356,-117.912135
5,1,26163596100,"(42.1014482, -83.1602786)",42.101448,-83.160279
9,1,9009165600,"(41.3558996, -72.9323558)",41.355900,-72.932356
10,3,36061001200,"(40.7159065, -73.9820936)",40.715907,-73.982094


## Display

In [8]:
df.to_csv('../LAR.csv')

In [ ]:
mapit = folium.Map(location=[37.09, -95.71], zoom_start=5, tiles="Stamen Toner")

for index, row in df.iterrows():
    if row['action_taken'] == 1:
        folium.CircleMarker( location=row['location'], color='#00FF00', fill_color='#00FF00', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )
    elif row['action_taken'] == 3:
        folium.CircleMarker( location=row['location'], color='#FF0000', fill_color='#FF0000', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )

mapit
